#Image Compression for Efficient Storage and Transmission

##LIBRARIES

In [6]:
import os
import numpy as np
import pywt
from PIL import Image
from io import BytesIO
import cv2

##jpeg Compression


In [7]:
def compress_jpeg(image_path, quality=10):
    image = Image.open(image_path)
    buffer = BytesIO()
    image.save(buffer, format="JPEG", quality=quality)
    return buffer.getvalue()

##Wavelet Compression Function (Using Haar Transform)

In [8]:
def compress_wavelet(image_path, wavelet='haar', level=2, threshold=30):
    image = Image.open(image_path).convert('L')  # Convert to grayscale
    img_array = np.array(image, dtype=np.float32)

    coeffs = pywt.wavedec2(img_array, wavelet=wavelet, level=level)
    coeffs_thresh = []

    for i, coeff in enumerate(coeffs):
        if i == 0:
            coeffs_thresh.append(coeff)
        else:
            cH, cV, cD = coeff
            cH = pywt.threshold(cH, threshold, mode='soft')
            cV = pywt.threshold(cV, threshold, mode='soft')
            cD = pywt.threshold(cD, threshold, mode='soft')
            coeffs_thresh.append((cH, cV, cD))

    img_reconstructed = pywt.waverec2(coeffs_thresh, wavelet)
    img_reconstructed = np.clip(img_reconstructed, 0, 255).astype(np.uint8)

    result_img = Image.fromarray(img_reconstructed)
    buffer = BytesIO()
    result_img.save(buffer, format="JPEG", quality=30)
    return buffer.getvalue()

##Run Compression On Image

In [11]:
input_path = "sample_2.jpg"  # Replace with your own image path if needed

jpeg_data = compress_jpeg(input_path)
wavelet_data = compress_wavelet(input_path)

##Save Compressed Outputs

In [12]:
with open("compressed_jpeg_2.jpg", "wb") as f:
    f.write(jpeg_data)

with open("compressed_wavelet_2.jpg", "wb") as f:
    f.write(wavelet_data)